# [训练一个中文GPT2](https://zhuanlan.zhihu.com/p/606339093)

## 加载包和处理数据
1. 我们只需要保证数据都含有content这一列
2. 使用`glob`包，获得所有的数据的路径
3. 我们使用`random`包，从所有的文件路径中，随机找50个数据路径，作为训练集合的使用

In [1]:
import random
from glob import glob
from datasets import load_dataset, DatasetDict

random.seed(44)  # 随机种子

all_file_list = glob(pathname="/root/dataset/gpt2_data/*/**")
test_file_list = random.sample(all_file_list, 50)
train_file_list = [i for i in all_file_list if i not in test_file_list]

len(train_file_list), len(test_file_list)

(9463, 50)

##  创建数据
1. 只要将路径放到一个字典里面。dict的key分别为`train`、`valid`，他们对应的值就是文件路径列表即可

In [2]:
raw_datasets = load_dataset("csv",data_files={'train':train_file_list,'valid':test_file_list}, cache_dir="cache_data")

raw_datasets

Resolving data files:   0%|          | 0/9463 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/50 [00:00<?, ?it/s]

Using custom data configuration default-3f54a3f7b8d40c50


Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to cache_data/csv/default-3f54a3f7b8d40c50/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['content', 'file_path'],
        num_rows: 9228698
    })
    valid: Dataset({
        features: ['content', 'file_path'],
        num_rows: 50527
    })
})

## Tokenizer
> 参考 [从头开始训练因果语言模型](https://huggingface.co/course/zh-CN/chapter7/6?fw=pt)。

1. Tokenizer 是最关键的一步，因为我们处理的是中文，因此使用`bert_base_chinese`就足够了
2. 如果你的语料里面有别的语言，你也可以使用多语言。这个都无所谓的。只要保证你使用的Tokenizer能覆盖你的数据即可
3. `context_length = 512`设置你的每一个文本的最长长度，我这里设置的是512，如果你的显卡显寸小，那你可以改小一点，比如128。但是多出来的数据，并不是说直接截断不要了，而是按照`context_length`长度，不断的对文本进行截断，大概就像是下面这样的：

<img src="https://huggingface.co/datasets/huggingface-course/documentation-images/resolve/main/en/chapter7/chunking_texts.svg"/>

4. 对于`gpt2`模型，需要告诉模型一句话从哪里开始，从哪里结束。因此我们需要设置`bos_token`、`eos_token`、`unk_token`



In [3]:
from transformers import AutoTokenizer, AutoConfig

context_length = 512
tokenizer = AutoTokenizer.from_pretrained("/root/model/bert-base-chinese")

# 测试 tokenizer
outputs = tokenizer(
    raw_datasets["train"][:2]["content"],
    truncation=True,
    max_length=context_length,
    return_overflowing_tokens=True,
    return_length=True,
)

print(f"Input IDs length: {len(outputs['input_ids'])}")
print(f"Input chunk lengths: {(outputs['length'])}")
print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 3
Input chunk lengths: [512, 253, 82]
Chunk mapping: [0, 0, 1]


In [4]:
tokenizer.add_special_tokens(special_tokens_dict={'bos_token': '<|endoftext|>',
 'eos_token': '<|endoftext|>',
 'unk_token': '<|endoftext|>'})

1

In [5]:
def tokenize(element):
    outputs = tokenizer(
        element["content"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


# 样本处理
tokenized_datasets = raw_datasets.map(
    tokenize, batched=True, remove_columns=raw_datasets["train"].column_names
)
tokenized_datasets

  0%|          | 0/9229 [00:00<?, ?ba/s]

  0%|          | 0/51 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 6175754
    })
    valid: Dataset({
        features: ['input_ids'],
        num_rows: 34382
    })
})

## 模型
1. 这里的`gpt2`模型，可不是使用别人训练好的，就是一个gpt2配置，因为我们要使用这个从头训练一个新的`gpt2`

In [6]:
from transformers import AutoTokenizer, GPT2LMHeadModel, AutoConfig

# config = AutoConfig.from_pretrained(
#     "gpt2",
#     vocab_size=len(tokenizer),
#     n_ctx=context_length,
#     bos_token_id=tokenizer.bos_token_id,
#     eos_token_id=tokenizer.eos_token_id,
# )
config = AutoConfig.from_pretrained(
    "/root/model/gpt2",
    vocab_size=len(tokenizer),
    n_ctx=context_length,
    bos_token_id=tokenizer.bos_token_id,
    eos_token_id=tokenizer.eos_token_id,
)

In [7]:
model = GPT2LMHeadModel(config)
model_size = sum(t.numel() for t in model.parameters())
print(f"GPT-2 size: {model_size/1000**2:.1f}M parameters")

GPT-2 size: 102.1M parameters


In [8]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [9]:
out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
for key in out:
    print(f"{key} shape: {out[key].shape}")

input_ids shape: torch.Size([5, 512])
attention_mask shape: torch.Size([5, 512])
labels shape: torch.Size([5, 512])


In [10]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="/root/model/gpt2/train/chinese_gpt2_big",
    per_device_train_batch_size=24,
    per_device_eval_batch_size=24,
    evaluation_strategy="steps",
    eval_steps=2_000,
    logging_steps=2_000,
    gradient_accumulation_steps=8,
    num_train_epochs=2,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=2_0000,
    fp16=True,
    push_to_hub=False,
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
)

Using amp fp16 backend


In [11]:
trainer.train()

***** Running training *****
  Num examples = 6175754
  Num Epochs = 2
  Instantaneous batch size per device = 24
  Total train batch size (w. parallel, distributed & accumulation) = 192
  Gradient Accumulation steps = 8
  Total optimization steps = 64330


Step,Training Loss,Validation Loss
2000,4.214000,2.988479
4000,2.933100,2.733274
6000,2.773500,2.630356
8000,2.688700,2.565998
10000,2.633400,2.518783
12000,2.593300,2.484987
14000,2.562500,2.457442
16000,2.537200,2.433167
18000,2.515000,2.413198
20000,2.495600,2.394620


***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
/usr/local/lib/python3.6/dist-packages/transformers/trainer.py:1310: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  args.max_grad_norm,
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation *****
  Num examples = 34382
  Batch size = 24
***** Running Evaluation

TrainOutput(global_step=64330, training_loss=2.4888972221480063, metrics={'train_runtime': 280050.64, 'train_samples_per_second': 44.105, 'train_steps_per_second': 0.23, 'total_flos': 3.227331287973888e+18, 'train_loss': 2.4888972221480063, 'epoch': 2.0})

In [12]:
import torch

# 保存模型
torch.save(model.state_dict(), '/root/model/gpt2/gpt2.bin')